In [ ]:
import minsearch
import json
import openai
from openai import OpenAI

In [42]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)
        
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)
index.fit(documents)

In [62]:
def minsearch(query):
    boost = { "question" : 3, "section" : 0.5 }

    results = index.search(
        query = query,
        filter_dict = {"course": "data-engineering-zoomcamp"},
        boost_dict = boost,
        num_results=5
    )

    return results

In [70]:
def build_prompt(query, search_results):
    prompt_template = """
        You're a course teaching assistant. 
        Answer the QUESTION based on the CONTEXT. 
        Use only the facts from the CONTEXT when answering the QUESTION.
        If the CONTEXT doesn't contain the answer, output IDK
        Answer in at most three sentences.
        
        QUESTION: {question}
        
        CONTEXT: 
        {context}
        
    """.strip()
    context = ""
    
    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [71]:
def llm(prompt):
    client = OpenAI()
        
    response = client.chat.completions.create(
        model = 'gpt-4o',
        messages = [{"role":"user", "content":prompt}]
    )

    return response.choices[0].message.content

In [72]:
def rag(query):
    results = minsearch(query)
    print(results)
    prompt = build_prompt(query, results)
    print(prompt)
    return llm(prompt)

In [73]:
query = 'course already started, can I still enroll?'
print(rag(query))

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'section': 'General course-related questions', 'question': 'Course - Can I follow the course after it finishes?', 'course': 'data-engineering-zoomcamp'}, {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anac